In [18]:
#libraries
import cv2
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile

In [19]:
# read the .xml-file
# zip_ref = zipfile.ZipFile('../CompanyMen_v1.0-split-012-Bobby_being_angry.azp')
zip_ref = zipfile.ZipFile('her_scene11_fuerChristian.azp')
zip_ref.extractall('/tmp')
tree = ET.parse('/tmp/content.xml')
root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
i=0
ts_end=[]
ts_begin=[]
for child in root[0].iter():
    if child.get('type')=='#Shot':
        i+=1
        for child2 in child:
            if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                ts_end.append(round(int(child2.get('end'))/1000*25))
                ts_begin.append(round(int(child2.get('begin'))/1000*25))

In [20]:
def read_target_colors_azp(azp_path):
    zip_ref = zipfile.ZipFile(azp_path)
    zip_ref.extractall('/tmp')
    tree = ET.parse('/tmp/content.xml')
    root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
    colors_target=[]
    for child in root[0].iter():
        if child.get('type')=='#ColourRange':
            for child2 in child:
                if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}content':
                    colors_target.append(child2.text.split(','))
    return colors_target

In [21]:
print('ts_begin: ',np.shape(ts_begin),ts_begin)
print('ts_end: ',np.shape(ts_end),ts_end)

ts_begin:  (23,) [0, 239, 465, 4436, 4510, 4602, 4651, 4717, 4813, 4930, 4974, 5017, 5247, 5278, 5546, 5643, 5684, 5730, 5795, 5846, 5953, 6008, 6043]
ts_end:  (23,) [239, 465, 4436, 4510, 4602, 4651, 4717, 4813, 4930, 4974, 5017, 5247, 5278, 5546, 5643, 5684, 5730, 5795, 5846, 5953, 6008, 6043, 6387]


In [22]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width,target_colorspace):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max length of the given segment
        while(vid.isOpened()):
            ret, frame = vid.read() # if ret is false, frame has no content
            if not ret:
                break
            if vid_length>=start_frame:
                frame=cv2.resize(frame,resolution) # resize the video to a different resolution
                frame=np.array(frame,dtype='float32')
                frames.append(frame) #add the individual frames to a list
                pbar.update(1) #update the progressbar
            if vid_length==end_frame:
                pbar.update(1)
                break
            vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    frames=change_colorspace(frames,target_colorspace)
    return frames[:-1]

In [23]:
def change_colorspace(frame_list,target_colorspace):
    changed_frame_list=[]
    if target_colorspace=='HSV':
        print('HSV')
        for frame in frame_list:
            frame=np.array(frame,dtype='uint8')
            frame=cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            frame=np.array(frame,dtype='float32')
            changed_frame_list.append(frame)
        return changed_frame_list
    if target_colorspace=='cie-lab':
        print('cie-lab')
        for frame in frame_list:
            frame=np.array(frame,dtype='uint8')
            frame=cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
            frame=np.array(frame,dtype='float32')
            changed_frame_list.append(frame)
        return changed_frame_list
    else:
        print('rgb')
        for frame in frame_list:
            frame=np.array(frame,dtype='uint8')
            frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame=np.array(frame,dtype='float32')
            changed_frame_list.append(frame)
        return changed_frame_list

In [24]:
def create_nns_picture(frame_list,target_colorspace,path):
    value_to_color=fn_value_to_color(target_colorspace,path) #get the color dict 
        
    color_value_list=[value for value in value_to_color]
    
    kdt = KDTree(color_value_list, leaf_size=30, metric='euclidean')  
    #flatten the image to 1d 
    img = frame_list.reshape((frame_list.shape[0] * frame_list.shape[1], 3))     

    nns = kdt.query(img, k=1, return_distance=False)
    
    changed_frame_aux=[]
    for nn in tqdm(nns):
        changed_frame_aux.append(color_value_list[nn[0]])
    changed_frame_aux=np.asarray(changed_frame_aux,dtype='uint8')
    changed_frame=changed_frame_aux.reshape(frame_list.shape[0],frame_list.shape[1],3)
    return changed_frame

In [30]:
def fn_value_to_color(target_colorspace,path):
            if (path != 'full'):
                print('Now using colors specified in path')
                colors_to_value_dict = {}
                with open(path) as f:
                    for line in f:
                        #split lines at "::
                        color, rgb = line.strip().split(':')
                        #strip the rgb-string of the parenthesis, split it up a the commas,
                        #cast them to int and put them into a tuples
                        rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                        colors_to_value_dict[color]=rgb_value
            else:
                colors_to_value_dict={
                'darkred':(139,0,0),
                'firebrick':(178,34,34),
                'crimson':(220,20,60),
                'red':(255,0,0),
                'tomato':(255,99,71),
                'salmon':(250,128,114),
                'darkorange':(255,140,0),
                'gold':(255,215,0),
                'darkkhaki':(189,183,107),
                'yellow':(255,255,0),
                'darkolivegreen':(85,107,47),
                'olivedrab':(107,142,35),
                'greenyellow':(173,255,47),
                'darkgreen':(0,100,0),
                'aquamarine':(127,255,212),
                'steelblue':(70,130,180),
                'skyblue':(135,206,235),
                'darkblue':(0,0,139),
                'blue':(0,0,255),
                'royalblue':(65,105,225),
                'purple':(128,0,128),
                'violet':(238,130,238),
                'deeppink':(255,20,147),
                'pink':(255,192,203),
                'antiquewhite':(250,235,215),
                'saddlebrown':(139,69,19),
                'sandybrown':(244,164,96),
                'ivory':(255,255,240),
                'dimgrey':(105,105,105),
                'grey':(128,128,128),
                'silver':(192,192,192),
                'lightgrey':(211,211,211),
                'black':(0,0,0),
                'white':(255,255,255),
                'darkcyan':(0,139,139),
                'cyan':(0,255,255),
                'green':(0,128,0),
                'khaki':(240,230,140),
                'goldenrod':(218,165,32),
                'orange':(255,165,0),
                'coral':(255,127,80),
                'magenta':(255,0,255),
                'wheat':(245,222,179),
                'skin':(255,224,189),
                'purple4':(147,112,219)}

            colors_aux={}
            if target_colorspace=='HSV':
                print('HSV')
                for color in colors_to_value_dict:
                    a = np.array((colors_to_value_dict[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2HSV)
                    c=np.array(c,dtype='float32')
                    colors_aux[color]=tuple(c.reshape(3))
                colors_to_value_dict=colors_aux

            if target_colorspace=='cie-lab':
                print('cie-lab')
                for color in colors_to_value_dict:
                    a = np.array((colors_to_value_dict[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2LAB)
                    c=np.array(c,dtype='float32')
                    colors_aux[color]=tuple(c.reshape(3))
                colors_to_value_dict=colors_aux


            value_to_color={}
            for color in colors_to_value_dict:
                value_to_color[colors_to_value_dict[color]]=color
            #purple4 is median purple
            #skin is caucasian        
            return value_to_color

In [26]:
resolution=1920
target_colorspace='cie-lab'
# video_path='../../Wells_John_CompanyMen_full.mp4'
video_path='Her_bluray_Szene 11_25fps.mp4'
scene_all_frames_lab=[]
for i,ts in enumerate(zip(ts_begin,ts_end)):
    scene_all_frames_lab.append(read_video_segments(video_path,
                                                    ts[0],ts[0]+2,resolution,target_colorspace))
    if i==5:
        break

# full_scene_all_frames_lab=[]
# for i,ts in enumerate(zip(ts_begin,ts_end)):
#     full_scene_all_frames_lab.append(read_video_segments(video_path,
#                                                     ts[0],ts[1],resolution,target_colorspace))
#     if i==5:
#         break

4it [00:00, 15.31it/s]                       
  0%|          | 0/3 [00:00<?, ?it/s]

cie-lab


4it [00:01,  3.57it/s]                       
  0%|          | 0/3 [00:00<?, ?it/s]

cie-lab


4it [00:01,  2.08it/s]                       
  0%|          | 0/3 [00:00<?, ?it/s]

...

cie-lab


4it [00:16,  4.07s/it]                       
  0%|          | 0/3 [00:00<?, ?it/s]

cie-lab


4it [00:17,  4.46s/it]                       

cie-lab


In [15]:
print('shape: ', np.shape(scene_all_frames_lab))
# print('shape: ', np.shape(full_scene_all_frames_lab))

shape:  (6, 2, 1080, 1920, 3)


In [31]:
# for i,frame_list in enumerate(scene_all_frames_lab):
#     #index=int(len(frame_list)/2)
#     try:
# #         frame=frame_list[index]
#         frame=create_nns_picture(frame_list[0],target_colorspace,'full')
#         cv2.imwrite('key_frames/ganzer_film/2_lab_full_1920w_'+str(i)+'.png',cv2.cvtColor(frame, cv2.COLOR_LAB2BGR))
#     except:
#         print('no valid timestamps at '+str(i))
#         pass

cie-lab


100%|██████████| 2073600/2073600 [00:00<00:00, 2147759.41it/s]


cie-lab


100%|██████████| 2073600/2073600 [00:00<00:00, 2233556.08it/s]


cie-lab


...

100%|██████████| 2073600/2073600 [00:00<00:00, 2122197.53it/s]


cie-lab


100%|██████████| 2073600/2073600 [00:00<00:00, 2114883.12it/s]


cie-lab


100%|██████████| 2073600/2073600 [00:00<00:00, 2164763.27it/s]


In [ ]:
def extract_dominant_colors(frame_list,target_colorspace,path,colors_to_return=5):
    print(str(len(frame_list))+' frames to process.')
    value_to_color=fn_value_to_color(target_colorspace,path) #get the color dict 

    bins={} #bins dict for histogram
    for value in value_to_color: #init the dict with ones for every key to avoid difficulties with divisions
                            # because the the sum of the bins goes from 500k to 2kk this shouldn't be a problem
        bins[value_to_color[value]]=1

    color_value_list=[value for value in value_to_color] #create a list of the color_values

    kdt = KDTree(color_value_list, leaf_size=30, metric='euclidean')
    for image in tqdm(frame_list): #traverse the video
        img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
        nns = kdt.query(img, k=1, return_distance=False)
        for nn in nns:
            bins[value_to_color[color_value_list[nn[0]]]]+=1

    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1] #normalize the bins
    bins_norm={k:v/norm_factor*10000 for k,v in bins.items()} #scale 0-10000 for visibility
    return bins_norm

In [ ]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe
    bins_sorted=list(zip(list(bins.values()),list(bins.keys())))
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
#     bin_threshold=bin_threshold/100 #scale the percentage to 0-1
#     df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df

In [ ]:
reduced_dataframes=[]
for shot in scene_all_frames_lab:
    bins=extract_dominant_colors(shot,target_colorspace,'colors')
    reduced_dataframes.append(bins_to_df(bins))

full_dataframes=[]
for shot in full_scene_all_frames_lab:
    bins=extract_dominant_colors(shot,target_colorspace,'full')
    full_dataframes.append(bins_to_df(bins))

In [ ]:
print(len(reduced_dataframes),len(full_dataframes))

In [ ]:
def process_df(reduced_dataframe_list,full_dataframe_list,ground_truth,noise_threshold):
    # create the lists that are returned
    # add the first entry of the histogramm to the output 
    # the 'real' output is the real_dataframe_list
    real_dataframe_list=[0]
    comparison=[0]
    
    # traverse the histograms
    # d is the current shot, d1 the following shot
    for i,(d,d1,e,e1) in enumerate(zip(reduced_dataframe_list,reduced_dataframe_list[1:],
                                       full_dataframe_list,full_dataframe_list[1:])):
        
        #calculate the absolute change of the histograms
        absolute_df=abs(d1-d) 
        absolute_df=absolute_df.sort_values(by='count',ascending=False)
        
        #apply a noise-filter to the absolute df
#         reduced_absolute_high=reduced_absolute_df[reduced_absolute_df>noise_threshold].dropna()
#         reduced_absolute_low=reduced_absolute_df[reduced_absolute_df<-noise_threshold].dropna()
#         reduced_absolute_denoised=reduced_absolute_high.combine_first(reduced_absolute_low)
        absolute_denoised=absolute_df[absolute_df>noise_threshold].dropna()
        
        # create the relative change, scaled to 0-100 percent
        # they are calculated from the denoised df, to prevent errors because of the noise
        relative_df=absolute_denoised/d*100
        relative_df=relative_df.dropna()
        relative_df=relative_df.sort_values(by='count',ascending=False)
        
        d_sorted=d.sort_values(by='count',ascending=False)
        
        #use the relative changes as weights for the absolute dataframe
        weighted_absolute_df=absolute_df*relative_df
        weighted_absolute_df=weighted_absolute_df.sort_values(by='count',ascending=False)

        # if there are no elements in the denoised df, it is assumed that all changes from
        # shot to shot are noise, in that case the current histogram is appended,
        # else the relative histogram
        if len(absolute_denoised)==0:
            real_dataframe_list.append(d_sorted)
        else:
            real_dataframe_list.append(relative_df)
        
        # create a comparison table
        relative=relative_df.head()
        relative=relative.drop('count',axis=1)
        relative=relative.reset_index(level=0,inplace=False)
        relative=relative.rename(index=str,columns={'color':'relative'})
        
        new=weighted_absolute_df.head()
        new=new.drop('count',axis=1)
        new=new.reset_index(level=0,inplace=False)
        new=new.rename(index=str,columns={'color':'reduced_new'})
        
        reduced_old=d_sorted.head()
        reduced_old=reduced_old.drop('count',axis=1)
        reduced_old=reduced_old.reset_index(level=0,inplace=False)
        reduced_old=reduced_old.rename(index=str,columns={'color':'reduced_old'})
        
        full_dd=e.sort_values(by='count',ascending=False)
        full_old=full_dd.head()
        full_old=full_old.drop('count',axis=1)
        full_old=full_old.reset_index(level=0,inplace=False)
        full_old=full_old.rename(index=str,columns={'color':'full_old'})

        gt=pd.DataFrame(ground_truth[i+1],columns=['ground_truth'])
        
        joined=pd.concat([full_old,reduced_old,new,relative,gt],axis=1)
        comparison.append(joined)
        
    return {'real': real_dataframe_list,'comparison':comparison}

In [ ]:
ground_truth=read_target_colors_azp('her_scene11_fuerChristian.azp')
new = process_df(reduced_dataframes,full_dataframes,ground_truth,100)

In [ ]:
new['comparison'][0]

In [ ]:
new['comparison'][1]

In [ ]:
new['comparison'][2]

In [ ]:
new['comparison'][3]

In [ ]:
new['comparison'][4]

In [ ]:
new['comparison'][5]

In [ ]:
pp=full_scene_all_frames_lab[3]
index=int(len(pp)/2)
# cv2.imwrite('key_frames/nn_by_hand.png',cv2.cvtColor(create_nns_picture(pp[index],target_colorspace,'full'), cv2.COLOR_LAB2BGR))
np.shape(pp)

In [ ]:
pixel_dict={'light_grey':pp[index][0,0], #coordinates in image are reversed 
           'grey':pp[index][43,96],
           'brown':pp[index][31.,107],
           'red':pp[index][111,0],
           'black':pp[index][1,198],
           'white':pp[index][74,54]}
pixel_dict

In [ ]:
min_dist=10000000
min_col=''
for c in lab_to_color:

    d = euclidean(c, np.asarray((6, 130, 129)))
    if d < min_dist:
        min_dist=d
        min_col=lab_to_color[c]
print(min_col,min_dist)

In [ ]:
lab_to_color=fn_rgb_to_color('cie-lab','full')
lab_list=[] #create a traverseable list of the rgb_values
for lab in lab_to_color: #map the values of the dict to a list
    lab_list.append(lab)
lab_to_color

In [ ]:
pixel_dict

In [ ]:
from scipy.spatial.distance import euclidean
import scipy
all_distances={}
color_to_color={}
for colorstr in pixel_dict:
    #distance={}
    dists=[]
    print("gt:",colorstr)

    for lab in lab_list:
        #print(lab,pixel_dict[pixel])
#         print(np.asarray(lab), lab_to_color[lab], colorstr, np.asarray(pixel_dict[colorstr]))
        cola = np.asarray(lab,dtype='float32')
        colb = np.asarray(pixel_dict[colorstr],dtype='float32')
        #print(cola,colb,euclidean(cola,colb))
#         print(euclidean(np.asarray((  0, 128, 128)) , np.asarray([  6, 130, 129],dtype='uint8')))
        dist=scipy.spatial.distance.euclidean(cola,colb)
        #print(dist)
        dists.append((lab_to_color[lab],dist))
        #distance[a]=lab
#     print((map(max,distance)))  
#     break
#    print("gt:",colorstr)
    print(min(dists, key = lambda t: t[1]))
#    print(dists)
    
    #all_distances[pixel]=distance
#     print(distance)
#     break
    #color_to_color[pixel]=min(distance.keys())#lab_to_color[distance[min(distance.keys())]]
#     break
#color_to_color

In [ ]:
from scipy.spatial.distance import euclidean
import scipy
euclidean(np.asarray((  0, 128, 128),dtype='uint8') , np.asarray([  6, 130, 129],dtype='uint8'))

In [ ]:
all_distances['black']

In [ ]:
a=[255,255,255]
b=[0,0,0]
euclidean(a,a)